In [1]:
import numpy as np
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt

In [2]:
structure = input("Enter the number of neurons of each layer in your network").split(" ")
structure = [int(x) for x in structure]
print(structure)

[4, 2, 2, 2, 2, 2, 2, 2, 2, 3]


In [3]:
data = load_iris()["data"]
target = load_iris()["target"]

In [71]:
data = data[[2, 57, 105, 15, 75, 125]]
target = target[[2, 57, 105, 15, 75, 125]]

In [4]:
for x, y in zip(data, target):
    print(x, y)

[5.1 3.5 1.4 0.2] 0
[4.9 3.  1.4 0.2] 0
[4.7 3.2 1.3 0.2] 0
[4.6 3.1 1.5 0.2] 0
[5.  3.6 1.4 0.2] 0
[5.4 3.9 1.7 0.4] 0
[4.6 3.4 1.4 0.3] 0
[5.  3.4 1.5 0.2] 0
[4.4 2.9 1.4 0.2] 0
[4.9 3.1 1.5 0.1] 0
[5.4 3.7 1.5 0.2] 0
[4.8 3.4 1.6 0.2] 0
[4.8 3.  1.4 0.1] 0
[4.3 3.  1.1 0.1] 0
[5.8 4.  1.2 0.2] 0
[5.7 4.4 1.5 0.4] 0
[5.4 3.9 1.3 0.4] 0
[5.1 3.5 1.4 0.3] 0
[5.7 3.8 1.7 0.3] 0
[5.1 3.8 1.5 0.3] 0
[5.4 3.4 1.7 0.2] 0
[5.1 3.7 1.5 0.4] 0
[4.6 3.6 1.  0.2] 0
[5.1 3.3 1.7 0.5] 0
[4.8 3.4 1.9 0.2] 0
[5.  3.  1.6 0.2] 0
[5.  3.4 1.6 0.4] 0
[5.2 3.5 1.5 0.2] 0
[5.2 3.4 1.4 0.2] 0
[4.7 3.2 1.6 0.2] 0
[4.8 3.1 1.6 0.2] 0
[5.4 3.4 1.5 0.4] 0
[5.2 4.1 1.5 0.1] 0
[5.5 4.2 1.4 0.2] 0
[4.9 3.1 1.5 0.2] 0
[5.  3.2 1.2 0.2] 0
[5.5 3.5 1.3 0.2] 0
[4.9 3.6 1.4 0.1] 0
[4.4 3.  1.3 0.2] 0
[5.1 3.4 1.5 0.2] 0
[5.  3.5 1.3 0.3] 0
[4.5 2.3 1.3 0.3] 0
[4.4 3.2 1.3 0.2] 0
[5.  3.5 1.6 0.6] 0
[5.1 3.8 1.9 0.4] 0
[4.8 3.  1.4 0.3] 0
[5.1 3.8 1.6 0.2] 0
[4.6 3.2 1.4 0.2] 0
[5.3 3.7 1.5 0.2] 0
[5.  3.3 1.4 0.2] 0


In [5]:
one_hot_labels = np.zeros((len(data), 3))
for i in range(len(one_hot_labels)):
    for j in range(len(one_hot_labels[i])):
        one_hot_labels[i, target[i]] = 1
print(one_hot_labels)

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0.

In [6]:
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

def sigmoid_derivative(x):
    return sigmoid(x) * (1.0 - sigmoid(x))

def softmax(x):
    exponential_sum = np.sum([np.exp(i) for i in x])
    exponential_x = np.array([np.exp(i) / exponential_sum for i in x])
    return exponential_x

def calculate_loss(actual, expected):
    loss = - np.sum(actual.reshape(-1) * expected)
    return loss

In [75]:
actual = np.array([[0.08], [0.24], [0.72]])
expected = np.array([0, 0, 1])
calculate_loss(actual, expected)

-0.72

In [11]:
H = len(structure) - 2
learning_rate = 0.10
epochs = 50
all_epoch_errors = []

for epoch in range(epochs):
    each_epoch_errors = []
    sum_losses = 0
    
    np.random.seed(2023)
    w_n = [np.random.randn(l, next_l) for l,next_l in zip(structure[:-1], structure[1:])] # (4,2), (2,3)
    b_n = [np.random.randn(l, 1) for l in structure[1:]]  # (2,1), (3,1)
    
    for k, (x, y) in enumerate(zip(data, one_hot_labels)):
        z_n, a_n = [], []
        
        for i, (b, w) in enumerate(zip(b_n, w_n)):
            if i == 0:
                z = np.dot(w.T, x).reshape(-1,1) + np.array(b)
                a = sigmoid(z)                
                
            elif (i > 0) and (i < H):
                z = np.dot(w.T, a)
                a = sigmoid(z)
            else:
                z = np.dot(w.T, a)
                a = softmax(z)
            
            """    
            print(z)
            print(a)
            print()
            """                
            z_n.append(z)
            a_n.append(a)
        loss = calculate_loss(a, y)
        #print(f"{epoch}-{k}th softmax loss : {loss}")
        sum_losses += loss
        # BackPropagate
        np.random.seed(2023)
        e_je_w_ns = [np.zeros(w.shape) for w in w_n]
        e_je_b_ns = [np.zeros(b.shape) for b in b_n]
                
        datapoint_errors = []
        
        for L in range(H, -1, -1):
            if L != H:
                delta = sigmoid_derivative(z_n[L]) * np.dot(w_n[L+1], delta)
            else:
                #print(y.reshape(-1,1).shape)
                delta = a_n[L] - y.reshape(-1, 1)   # (3,1)                
            e_je_b_ns[L] = delta
            
            if L != 0:
                e_je_w_ns[L] = np.dot(a_n[L-1], delta.T)                
                #print(f"delta.T shape : {delta.T.shape}")
                #print(f"a_n[L] shape : {a_n[L].shape}")                
            else:
                #print(f"x : {x.reshape(-1,1).shape}")
                #print(f"delta : {delta.T.shape}")
                e_je_w_ns[L] = np.dot(x.reshape(-1,1), delta.T)   # (4,1), (1,2), (4,2)          
        """      
        print(f"{epoch}epoch {k}th data e_je_w_ns :\n {e_je_w_ns}")
        print(f"{epoch}epoch {k}th data e_je_b_ns :\n {e_je_b_ns}")
        print()
        """
        

0th loss : -0.7961057810106545
0th loss : -1.5922143001887443
0th loss : -2.388321318154082
0th loss : -3.184427177285007
0th loss : -3.980531532889161
0th loss : -4.776639693621629
0th loss : -5.572747223809919
0th loss : -6.368852621750726
0th loss : -7.164959832233739
0th loss : -7.961063978192976
0th loss : -8.757168922191639
0th loss : -9.553272535833743
0th loss : -10.349377659665192
0th loss : -11.145482295583912
0th loss : -11.941588918458551
0th loss : -12.737696233617797
0th loss : -13.533807340851874
0th loss : -14.329916242857099
0th loss : -15.126023475316751
0th loss : -15.922129383121902
0th loss : -16.71823541295486
0th loss : -17.51434518386445
0th loss : -18.31045058651341
0th loss : -19.106562962630733
0th loss : -19.90266466066792
0th loss : -20.698772275208228
0th loss : -21.49488265708078
0th loss : -22.29098823901182
0th loss : -23.087095268875117
0th loss : -23.88320011501899
0th loss : -24.679306251610775
0th loss : -25.47541811958662
0th loss : -26.27151355179

In [27]:
z_n

[array([[ 6.27374581],
        [-6.78636791]]),
 array([[ 0.09834887],
        [-3.11682648],
        [-1.07497685]])]

In [28]:
a_n

[array([[0.99811839],
        [0.00112779]]),
 array([[0.74102356],
        [0.02975083],
        [0.22922561]])]

In [29]:
a0, a1, a2 = np.exp(0.09834887), np.exp(-3.11682648), np.exp(-1.07497685)
a = a0 + a1 + a2
print(a0, a1, a2)
print(a0/a, a1/a, a2/a)

1.1033476428641078 0.04429752467070435 0.34130565643548416
0.7410235617601022 0.029750831227981552 0.22922560701191635


In [30]:
softmax(z_n[1])

array([[0.74102356],
       [0.02975083],
       [0.22922561]])